In [11]:
import pandas as pd
from openpyxl import load_workbook

    #database
df_ffca= pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0222\FFCA.xlsx' )
df_qv = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0222\QV.xlsx')
df_stock = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0222\stock.xlsx') 
df_stock.drop([0])
df_inventory = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0222\inventory.xlsx')
df_WC = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0222\fixed_doc\Warehouse_Risk_Category.xlsx')
df_CS = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0222\CS.xlsx')
df_CP = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0222\fixed_doc\CP_approved.xlsx')
df_movement = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0222\movement.xlsx')
df_vessel = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0222\inventory2.xlsx')
df_invoice = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0222\invoice.xlsx')


In [13]:
df_input= pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0222\input.xlsx')
df_invoice = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0222\invoice.xlsx')

def sales(a):
    return a == "sales"
df4 = df_input.loc[df_input['status'].apply(sales)]

In [114]:
#column filter
cols_ffca = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity', 'Inventory ID','Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
   'Purchase Settled Amount', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value', 'Treasury Payment Date', 'Purchase Invoice', 'Purchase Invoiced Amount', \
      'Sales Invoiced Amount', 'Sales Settled Amount', 'QTY']
cols_qv = ['Assignment ID', 'Group Company', 'Trade ID', 'Quota ID', 'Traffic Operator']
cols_inventory = ['INVENTORY ID', 'PURCHASE ASSIGNMENT', 'DRY WEIGHT', 'WET WEIGHT', 'EXPECTED RECEIPT DATE']
cols_stock = ['Inventory Id','Origin Country']
cols_cs = ['ID', 'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank', 'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount']
cols_ffca_blend = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity', 'Inventory ID','Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
   'Purchase Settled Amount', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value', 'Treasury Payment Date', 'Purchase Invoice', 'Purchase Invoiced Amount', \
      'Sales Invoiced Amount', 'Sales Settled Amount', 'QTY']
cols_invoice = ['Assignment Ref', 'Invoice Type', 'Invoice Date', 'Payment Expected Due Date', 'Days Overdue', 'Invoice No', 'Unpaid Amount USD', 'Invoice Amount']

df_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca)
df_qv_f1 = pd.DataFrame(df_qv, columns=cols_qv)
df_inventory_f1 = pd.DataFrame(df_inventory, columns=cols_inventory)
df_stock_f1 = pd.DataFrame(df_stock, columns=cols_stock)
df_CS_f1 = pd.DataFrame(df_CS, columns=cols_cs)
df_invoice_f1 = pd.DataFrame(df_invoice, columns=cols_invoice)
df_ffca_f1['Current Value'] = pd.to_numeric(df_ffca_f1['Current Value'], errors='coerce')
df_ffca_f1['Authorised Paid Quantity'] = pd.to_numeric(df_ffca_f1['Authorised Paid Quantity'], errors='coerce')
df_ffca_f1['Financed Quantity'] = pd.to_numeric(df_ffca_f1['Financed Quantity'], errors='coerce')


#sales
#delete nan sales
df_ffca_f1.dropna(subset=['Sales Assignment ID '], inplace=True)
# 一、先将字段拆分
df_CS_f1['Assignments']=df_CS['Assignments'].map(lambda x:x.split(','))
#二、然后直接调用explode()方法
df_CS_f1_explode = df_CS_f1.explode('Assignments')

def blend(a):
   return a == "Blend"

#blend
bdf_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca_blend)
df_blend_inventory = df_inventory_f1.loc[df_inventory_f1['PURCHASE ASSIGNMENT'].apply(blend)]

row_list = df_inventory_f1[df_inventory_f1['PURCHASE ASSIGNMENT'] == 'Blend'].index.tolist() # 获得含有该值的行的行号
df_inventory_f1 = df_inventory_f1.drop(row_list)

bffca_qv = pd.merge(bdf_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'left')
bffca_qv_cp = pd.merge(bffca_qv, df_CP, left_on=['Counterparty of Sales'], right_on=['Sales Assignment Counterparty'], how='outer')
binventory_stock = pd.merge(df_blend_inventory, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'], how='left')
FQC = pd.merge(bffca_qv_cp, df_CS_f1_explode, left_on=["Sales Assignment ID "], right_on=["Assignments"], how = 'outer')
bfinal_sales = pd.merge(FQC, binventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])

def function(a, b, c):
   return c * (1 + (a-b)/a)

bfinal_sales['Purchased Wet Weight']  = bfinal_sales.apply(lambda x: function(x['WET WEIGHT'], x['DRY WEIGHT'], x['QTY']), axis = 1)

bfinal_sales = bfinal_sales.drop(columns = ['WET WEIGHT', 'DRY WEIGHT'])

#non-blend
ffca_qv = pd.merge(df_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'left')
ffca_qv_cp = pd.merge(ffca_qv, df_CP, left_on=['Counterparty of Sales'], right_on=['Sales Assignment Counterparty'], how='outer')
inventory_stock = pd.merge(df_inventory_f1, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'], how='left')
nFQC = pd.merge(ffca_qv_cp, df_CS_f1_explode, left_on=["Sales Assignment ID "], right_on=["Assignments"], how = 'outer')
nfinal_sales = pd.merge(nFQC, inventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])
nfinal_sales.columns = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity',
   'Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales',
   'Sales Invoice ID', 'Estimated Sales Valuation',
   'Purchase Settled Amount', 'Authorised Paid Quantity',
   'Financed Quantity', 'Current Value', 'Treasury Payment Date',
   'Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount',
   'Sales Settled Amount', 'QTY', 'Assignment ID', 'Group Company',
   'Trade ID', 'Quota ID', 'Traffic Operator',
   'Sales Assignment Counterparty',
   'Sales Contractual Counterparty Open Account Approved ', 'ID',
   'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank',
   'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount',
   'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Purchase Dry Weight', 'Purchase Wet Weight',
   'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country']

order = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity','Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
   'Purchase Settled Amount', 'Authorised Paid Quantity','Financed Quantity', 'Current Value', 'Treasury Payment Date','Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount','Sales Settled Amount',  \
      'Assignment ID', 'Group Company','Trade ID', 'Quota ID', 'Traffic Operator','Sales Assignment Counterparty','Sales Contractual Counterparty Open Account Approved ', 'ID', \
   'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank','Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount', 'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'QTY', 'Purchased Wet Weight',\
   'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country']

bfinal_sales = bfinal_sales[order]

bfinal_sales.columns = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity',
   'Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales',
   'Sales Invoice ID', 'Estimated Sales Valuation',
   'Purchase Settled Amount', 'Authorised Paid Quantity',
   'Financed Quantity', 'Current Value', 'Treasury Payment Date',
   'Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount',
   'Sales Settled Amount', 'Assignment ID', 'Group Company', 'Trade ID',
   'Quota ID', 'Traffic Operator', 'Sales Assignment Counterparty',
   'Sales Contractual Counterparty Open Account Approved ', 'ID',
   'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank',
   'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount',
   'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Purchase Dry Weight', 'Purchase Wet Weight',
   'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country']

final_sales = pd.concat([nfinal_sales,bfinal_sales])

final_sales = pd.merge(final_sales, df_invoice_f1, left_on=["Sales Assignment ID "], right_on=["Assignment Ref"], how='left')
final_sales.fillna(0)



,Purchase Assignment ID,Sales Assignment ID,Commodity,Inventory ID,Counterparty of Purchase,Counterparty of Sales,Sales Invoice ID,Estimated Sales Valuation,Purchase Settled Amount,Authorised Paid Quantity,...,Inventory Id,Origin Country,Assignment Ref,Invoice Type,Invoice Date,Payment Expected Due Date,Days Overdue,Invoice No,Unpaid Amount USD,Invoice Amount
0,652590.5.12,671515.1.1,Nickel Concentrates,673961.0,Zopco SA,"Cd Logistics Qingdao Co., Ltd.",PTESI1135756,162163,-106356.37,48.0,...,673961.0,Zimbabwe,0,0,0,0,0,0,0,0.00
1,652590.5.13,671515.1.2,Nickel Concentrates,673962.0,Zopco SA,"Cd Logistics Qingdao Co., Ltd.",PTESI1144895,2.69565e+06,-1767109.43,797.0,...,673962.0,Zimbabwe,0,0,0,0,0,0,0,0.00
2,672522.2.6,670300.3.3,Nickel Intermediates,804529.0,Terrafame Oy,"Cd Logistics Qingdao Co., Ltd.",PTESI1164943,4.3387e+06,-4637243.74,400.0,...,804529.0,Finland,0,0,0,0,0,0,0,0.00
3,650041.5.14,670300.2.1,Nickel Intermediates,743524.0,Trafigura Ventures V B.V.,"Cd Logistics Qingdao Co., Ltd.",PTESI1160188,4.72918e+06,-3219461.65,438.0,...,743524.0,Finland,670300.2.1,FINAL,06-Dec-22,2023-01-30 00:00:00,17,PTESI1160188,-242506,-242505.91
4,672522.1.2,670300.2.2,Nickel Intermediates,747552.0,Terrafame Oy,"Cd Logistics Qingdao Co., Ltd.",PTESI1160189,7.17357e+06,-9755732.44,690.0,...,747552.0,Finland,670300.2.2,FINAL,06-Dec-22,2023-01-30 00:00:00,17,PTESI1160189,-179987,-179987.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24837,655810.9.8,677678.1.1,Nickel Intermediates,831140.0,Trafigura Nat Gas Limited,Vale Newfoundland & Labrador Limited,PTESI1153163,8.54381e+07,-7918539.46,847.0,...,831140.0,Finland,677678.1.1,ServiceInvoice,03-Feb-23,2023-03-03 00:00:00,-,494465,-3215.52,-3215.52
24838,655810.6.4,677678.1.1,Nickel Intermediates,831140.0,Trafigura Nat Gas Limited,Vale Newfoundland & Labrador Limited,PTESI1153163,8.54381e+07,-6154434.17,891.0,...,831140.0,Finland,677678.1.1,ServiceInvoice,03-Feb-23,2023-03-03 00:00:00,-,494465,-3215.52,-3215.52
24839,655810.5.10,677678.1.1,Nickel Intermediates,831140.0,Trafigura Nat Gas Limited,Vale Newfoundland & Labrador Limited,PTESI1153163,8.54381e+07,-7905639.41,903.0,...,831140.0,Finland,677678.1.1,ServiceInvoice,03-Feb-23,2023-03-03 00:00:00,-,494465,-3215.52,-3215.52
24840,655810.6.6,677678.1.1,Nickel Intermediates,831140.0,Trafigura Nat Gas Limited,Vale Newfoundland & Labrador Limited,PTESI1153163,8.54381e+07,-11783343.36,1343.0,...,831140.0,Finland,677678.1.1,ServiceInvoice,03-Feb-23,2023-03-03 00:00:00,-,494465,-3215.52,-3215.52


In [115]:
final_sales['Unpaid Amount USD'] = pd.to_numeric(final_sales['Unpaid Amount USD'], errors='coerce')
final_sales['Estimated Sales Valuation'] = pd.to_numeric(final_sales['Invoice Amount'], errors='coerce')
final_sales['Current Value'] = pd.to_numeric(final_sales['Current Value'], errors='coerce')

#Comparison between invoice amount and current value


def function1(a,b):
   if a == 'PROVISIONAL':
      return b
   else:
      return 0


def function2(a,b):
   if a == 'FINAL':
      return b
   else:
      return 0

final_sales['Unpaid Provisional Receivables(USD)'] = final_sales.apply(lambda x: function1(x['Invoice Type'], x['Unpaid Amount USD']), axis = 1)
final_sales['Unpaid Final Receivables(USD)'] = final_sales.apply(lambda x: function2(x['Invoice Type'], x['Unpaid Amount USD']), axis = 1) #wrong
final_sales.loc[:,"Uninvoiced Final Receivables(USD)"] = final_sales["Estimated Sales Valuation"] - final_sales["Invoice Amount"]


def function3(a,b):
   if -a >= b:
      return b
   if -a < b:
      return -a

def function4(a,b):
   if -a < b:
      return b + a
   if -a >= b:
      return 0

def function5(a,b):
   if a >= b:
      return b
   else:
      return a

def function6(a):
   if a == "FINAL":
      return "100%"
   else:
      return "90%"

final_sales['Total Financeable Sales Invoices Covered by LC']  = final_sales.apply(lambda x: function3(x['Invoice Amount'], x['Balance Amount']), axis = 1)
final_sales['LC Cover Available to SCR (SCR-Sales Contractual Rights)']  = final_sales.apply(lambda x: function4(x['Invoice Amount'], x['Balance Amount']), axis = 1)
final_sales['Sales Provisional Percentage']  = final_sales.apply(lambda x: function6(x['Invoice Type']), axis = 1)
final_sales['SCR Covered By LC (SCR-Sales Contractual Rights)']  = final_sales.apply(lambda x: function5(x['LC Cover Available to SCR (SCR-Sales Contractual Rights)'], x['Uninvoiced Final Receivables(USD)']), axis = 1)


import numpy as np

#adding 
final_sales['Sales Provisional Number'] = '1'
final_sales['Authorised Paid Quantity'] = pd.to_numeric(final_sales['Authorised Paid Quantity'], errors='coerce')
final_sales['Financed Quantity'] = pd.to_numeric(final_sales['Financed Quantity'], errors='coerce')
final_sales.loc[:,"Financing Ratio"] = final_sales["Authorised Paid Quantity"]/final_sales["Financed Quantity"]

final_sales['Cost Code ID'] = 'I01'
final_sales['Invoice Line Quantity'] = '0'
final_sales['Invoice Line Quantity UOM'] = 'MT'
final_sales['Purchase Dry Weight UOM'] = 'MT'
final_sales['Purchase Wet Weight UOM'] = 'MT'
final_sales['Receipt In Progress'] = 'N'
final_sales['Delivery In Progress'] = 'N'
final_sales['Summary Outstanding Receivables(USD)'] = final_sales['Unpaid Provisional Receivables(USD)'] + final_sales['Unpaid Final Receivables(USD)'] + final_sales['Uninvoiced Final Receivables(USD)']
final_sales['Sales Invoice Counterparty'] = final_sales['Counterparty of Sales']
final_sales['Eligible Dry Weight'] = final_sales['Purchase Dry Weight']
final_sales['Eligible Wet Weight'] = final_sales['Purchase Wet Weight']
final_sales['Paid Amount'] = final_sales['Invoice Amount'] - final_sales['Unpaid Amount USD']



def function7(a, b):
   if b == 0:
      return 0
   else:
      return -a/b
      
def function8(a):
   if a>1:
      return 1
   else:
      return a
      


final_sales['Sales Assignment Eligible Percentage']  = final_sales.apply(lambda x: function7(x['Estimated Sales Valuation'], x['Invoice Amount']), axis = 1)

final_sales['Sales Assignment Eligible Percentage']  = final_sales.apply(lambda x: function8(x['Sales Assignment Eligible Percentage']), axis = 1)

final_sales = final_sales.drop(columns=['Inventory ID', 'Current Value','Purchase Settled Amount', 'Purchase Invoice', 'Purchase Invoiced Amount', 'Current Value', 'Treasury Payment Date', \
'Assignment ID', 'Sales Assignment Counterparty', 'Assignments', 'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Unpaid Amount USD', \
   'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country', 'QTY', 'Assignment Ref', 'Sales Invoice ID', 'Sales Invoiced Amount', 'Sales Settled Amount'], axis=1)


final_sales.columns = ['Purchase Assignment Reference', 'Sales Assignment Reference', 'Purchase Commodity',
   'Purchase Contractual Counterparty', 'Sales Assignment Counterparty', 
   'Cost Amount(USD)', 'Authorised Paid Quantity','Financed Quantity', 
   'Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Traffic Operator',
   'Sales Contractual Counterparty Open Account Approved', 'LC Reference', 'Sales Credit Security Type',
   'LC Opening Bank', 'LC Confirming Bank', 'LC Issuance Date', 'LC Expiry Date', 'LC Current Balance', 'LC Original Amount',
   'Purchase Dry Weight', 'Purchase Wet Weight', 
   'Sales Provisional Final', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue','Sales Invoice ID','Invoice Line Amount(USD)', 'Unpaid Provisional Receivables(USD)',
   'Unpaid Final Receivables(USD)', 'Uninvoiced Final Receivables(USD)',
   'Total Financeable Sales Invoices Covered by LC',
   'LC Cover Available to SCR (SCR-Sales Contractual Rights)',
   'Sales Provisional Percentage',
   'SCR Covered By LC (SCR-Sales Contractual Rights)',
   'Sales Provisional Number', 'Financing Ratio', 'Cost Code ID',
   'Invoice Line Quantity', 'Invoice Line Quantity UOM',
   'Purchase Dry Weight UOM', 'Purchase Wet Weight UOM',
   'Receipt In Progress', 'Delivery In Progress',
   'Summary Outstanding Receivables(USD)', 'Sales Invoice Counterparty',
   'Eligible Dry Weight', 'Eligible Wet Weight','Paid Amount',
   'Sales Assignment Eligible Percentage']


order4 = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Sales Assignment Reference','Purchase Traffic Operator', \
   'Purchase Contractual Counterparty', 'Purchase Commodity', 'Sales Assignment Counterparty', 'Sales Credit Security Type', 'LC Reference', 'LC Original Amount', \
      'LC Current Balance', 'LC Issuance Date', 'LC Expiry Date', 'LC Opening Bank', 'LC Confirming Bank', 'Total Financeable Sales Invoices Covered by LC', \
            'LC Cover Available to SCR (SCR-Sales Contractual Rights)', 'SCR Covered By LC (SCR-Sales Contractual Rights)', 'Sales Invoice ID', 'Sales Provisional Final', \
               'Sales Provisional Number', 'Sales Provisional Percentage', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue', 'Sales Invoice Counterparty', \
                  'Cost Code ID', 'Cost Amount(USD)', 'Invoice Line Quantity', 'Invoice Line Quantity UOM', 'Invoice Line Amount(USD)', 'Paid Amount', 'Sales Assignment Eligible Percentage', \
                        'Authorised Paid Quantity', 'Financed Quantity', 'Financing Ratio', 'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(USD)', 'Uninvoiced Final Receivables(USD)', \
                           'Summary Outstanding Receivables(USD)', 'Sales Contractual Counterparty Open Account Approved', 'Purchase Dry Weight', 'Purchase Dry Weight UOM', \
                              'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight']

final_sales_test = final_sales[order4]



In [116]:
result4 = pd.merge(final_sales_test, df4, on=["Purchase Assignment Reference"], how='right')
result4['Cost Amount(USD)'] = pd.to_numeric(result4['Cost Amount(USD)'], errors='coerce')
result4['Paid Amount'] = pd.to_numeric(result4['Paid Amount'], errors='coerce')
result4 = result4.drop_duplicates(['Purchase Assignment Reference', 'Sales Invoice ID'])

result4['%'] = (result4['Purchase Dry Weight']/result4.groupby('Sales Invoice ID')['Purchase Dry Weight'].transform('sum'))
result4['Unpaid Provisional Receivables(USD)'] = result4['Unpaid Provisional Receivables(USD)'] * result4['%']
result4['Unpaid Final Receivables(USD)'] = result4['Unpaid Final Receivables(USD)'] * result4['%']



In [117]:
dfa = result4.groupby(['Purchase Assignment Reference', 'Sales Assignment Reference'])['Invoice Line Amount(USD)'].sum()
dfb = result4.groupby(['Purchase Assignment Reference', 'Sales Assignment Reference'])['Cost Amount(USD)'].sum()
dfc = result4.groupby(['Purchase Assignment Reference', 'Sales Assignment Reference'])['Paid Amount'].sum()
dfd = result4.groupby(['Purchase Assignment Reference', 'Sales Assignment Reference'])['Unpaid Provisional Receivables(USD)'].sum()
dfe = result4.groupby(['Purchase Assignment Reference', 'Sales Assignment Reference'])['Unpaid Final Receivables(USD)'].sum()
 
dfa.reset_index(drop=False)
dfb.reset_index(drop=False)
dfc.reset_index(drop=False)
dfd.reset_index(drop=False)
dfe.reset_index(drop=False)


df_1 = pd.merge(dfa,dfb, on = 'Sales Assignment Reference')
df_2 = pd.merge(dfd,dfc, on = 'Sales Assignment Reference')
df_3 = pd.merge(df_2,dfe, on = 'Sales Assignment Reference')
df_4 = pd.merge(df_2,df_3, on = 'Sales Assignment Reference')
df_5 = pd.merge(df_4,df_1, on = 'Sales Assignment Reference')


df5 = df_5.reset_index()
df5

order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Sales Assignment Reference','Purchase Traffic Operator', \
'Purchase Contractual Counterparty', 'Purchase Commodity', 'Sales Assignment Counterparty', 'Sales Credit Security Type', 'LC Reference', 'LC Original Amount', \
   'LC Current Balance', 'LC Issuance Date', 'LC Expiry Date', 'LC Opening Bank', 'LC Confirming Bank', 'Total Financeable Sales Invoices Covered by LC', \
      'LC Cover Available to SCR (SCR-Sales Contractual Rights)', 'SCR Covered By LC (SCR-Sales Contractual Rights)', 'Sales Invoice ID', 'Sales Provisional Final', \
            'Sales Provisional Number', 'Sales Provisional Percentage', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue', 'Sales Invoice Counterparty', \
               'Cost Code ID', 'Cost Amount(USD)', 'Invoice Line Quantity', 'Invoice Line Quantity UOM', 'Invoice Line Amount(USD)', 'Paid Amount', 'Sales Assignment Eligible Percentage', \
                  'Authorised Paid Quantity', 'Financed Quantity', 'Financing Ratio', 'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(USD)', 'Uninvoiced Final Receivables(USD)', \
                        'Summary Outstanding Receivables(USD)', 'Sales Contractual Counterparty Open Account Approved', 'Purchase Dry Weight', 'Purchase Dry Weight UOM', \
                           'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight']
                           
dff = pd.merge(df5, result4, on='Sales Assignment Reference')
dff = dff.drop(columns = ['Cost Amount(USD)_y', 'Paid Amount_y', 'Invoice Line Amount(USD)_y', 'Unpaid Final Receivables(USD)_y'])
dff = dff.drop_duplicates(['Purchase Assignment Reference'])
dff = dff.rename(columns={'Cost Amount(USD)_x':'Cost Amount(USD)', 'Paid Amount_x':'Paid Amount', 'Invoice Line Amount(USD)_x':'Invoice Line Amount(USD)', 'Unpaid Final Receivables(USD)_x':'Unpaid Final Receivables(USD)'})
dff = dff [order]

import numpy as np

dff['Sales Invoice ID'] = np.nan
dff['Sales Provisional Final '] = np.nan
dff['Sales Provisional Number'] = np.nan
dff['Sales Provisional Percentage'] = np.nan
dff['Sales Invoice Date'] = np.nan
dff['Sales Invoice Due Date'] = np.nan
dff['Sales Invoice Days Overdue'] = np.nan
dff['Sales Invoice Counterparty'] = np.nan
dff['Cost Code ID'] = np.nan
dff['Invoice Line Quantity'] = np.nan
dff['Invoice Line Quantity UOM'] = np.nan

dff['Total Financeable Sales Invoices Covered by LC']  = dff.apply(lambda x: function3(x['Invoice Line Amount(USD)'], x['LC Current Balance']), axis = 1)
dff['LC Cover Available to SCR (SCR-Sales Contractual Rights)']  = dff.apply(lambda x: function4(x['Invoice Line Amount(USD)'], x['LC Current Balance']), axis = 1)
dff['Sales Provisional Percentage']  = dff.apply(lambda x: function6(x['Sales Provisional Final ']), axis = 1)
dff['LC Cover Available to SCR (SCR-Sales Contractual Rights)'] = pd.to_numeric(dff['LC Cover Available to SCR (SCR-Sales Contractual Rights)'], errors='coerce') #chenge to float
dff['SCR Covered By LC (SCR-Sales Contractual Rights)']  = dff.apply(lambda x: function5(x['LC Cover Available to SCR (SCR-Sales Contractual Rights)'], x['Uninvoiced Final Receivables(USD)']), axis = 1)
dff = dff.rename(columns={'Sales Provisional Final ':'Sales Provisional Final', })

dff = dff[order4]
dff.reset_index(drop=True)


,Purchase Group Company Code,Purchase Trade ID,Purchase Quota Titan ID,Purchase Assignment Reference,Sales Assignment Reference,Purchase Traffic Operator,Purchase Contractual Counterparty,Purchase Commodity,Sales Assignment Counterparty,Sales Credit Security Type,...,Summary Outstanding Receivables(USD),Sales Contractual Counterparty Open Account Approved,Purchase Dry Weight,Purchase Dry Weight UOM,Purchase Wet Weight,Purchase Wet Weight UOM,Receipt In Progress,Delivery In Progress,Eligible Dry Weight,Eligible Wet Weight
0,NaN,NaN,NaN,647556.32.2,671366.36.2,NaN,Zangezur Copper-Molybdenum Combine CJSC,Copper Concentrates,Yunnan Copper H.K Limited,Documentary Letter of Credit,...,NaN,N,1507.624648,MT,1650.000000,MT,N,N,1507.624648,1650.000000
1,NaN,NaN,NaN,647556.32.3,671366.36.3,NaN,Zangezur Copper-Molybdenum Combine CJSC,Copper Concentrates,Yunnan Copper H.K Limited,NaN,...,NaN,N,1507.624648,MT,1650.000000,MT,N,N,1507.624648,1650.000000
2,NaN,NaN,NaN,647556.32.5,671366.36.5,NaN,Zangezur Copper-Molybdenum Combine CJSC,Copper Concentrates,Yunnan Copper H.K Limited,NaN,...,NaN,N,1507.624648,MT,1650.000000,MT,N,N,1507.624648,1650.000000
3,NaN,NaN,NaN,647556.32.6,671366.36.6,NaN,Zangezur Copper-Molybdenum Combine CJSC,Copper Concentrates,Yunnan Copper H.K Limited,NaN,...,NaN,N,1579.284177,MT,1728.426832,MT,N,N,1579.284177,1728.426832
4,NaN,NaN,NaN,647760.31.2,671366.38.1,NaN,Codelco Chile,Copper Concentrates,Yunnan Copper H.K Limited,Documentary Letter of Credit,...,NaN,N,10136.956000,MT,11003.000000,MT,N,N,10136.956000,11003.000000
5,PTE,649025.0,649025.80,649025.8.4,648918.35.3,Aditya Sharma,Ceylan Grup Metal Oksit Kimya Uretim Makine Hurda,Zinc Concentrates,Nyrstar Sales & Marketing SA,NaN,...,NaN,Y,334.501487,MT,360.960525,MT,N,N,334.501487,360.960525
6,PTE,649921.0,649921.37,649921.37.2,670638.1.8,Alkistis Korali,Simar (Societa Metalli Marghera S.P.A.),Copper Blister,"Wuzhou Jinsheng Copper Co., Ltd",NaN,...,3925727.49,N,243.340000,MT,243.340000,MT,N,N,243.340000,243.340000
7,PTE,649921.0,649921.37,649921.37.3,670638.1.8,Alkistis Korali,Simar (Societa Metalli Marghera S.P.A.),Copper Blister,"Wuzhou Jinsheng Copper Co., Ltd",NaN,...,3925727.49,N,243.560000,MT,243.560000,MT,N,N,243.560000,243.560000
8,PTE,649921.0,649921.37,649921.37.4,671361.1.2,Alkistis Korali,Simar (Societa Metalli Marghera S.P.A.),Copper Blister,"Wuzhou Jinsheng Copper Co., Ltd",NaN,...,4368344.31,N,244.120000,MT,244.120000,MT,N,N,244.120000,244.120000
9,PTE,649921.0,649921.37,649921.37.5,671361.1.2,Alkistis Korali,Simar (Societa Metalli Marghera S.P.A.),Copper Blister,"Wuzhou Jinsheng Copper Co., Ltd",NaN,...,4368344.31,N,244.160000,MT,244.160000,MT,N,N,244.160000,244.160000


In [120]:
result4['LC Current Balance'] = np.nan
result4['Sales Credit Security Type'] = np.nan
result4['LC Reference'] = np.nan
result4['LC Original Amount'] = np.nan
result4['LC Issuance Date'] = np.nan
result4['LC Expiry Date'] = np.nan
result4['LC Confirming Bank'] = np.nan
result4['Total Financeable Sales Invoices Covered by LC'] = np.nan
result4['LC Cover Available to SCR (SCR-Sales Contractual Rights)'] = np.nan
result4['SCR Covered By LC (SCR-Sales Contractual Rights)'] = np.nan
result4['Cost Amount(USD)'] = np.nan
result4['Authorized Paid Quantity'] = np.nan
result4['Financed Quantity'] = np.nan
result4['Uninvoiced Final Receivables(USD)'] = np.nan
result4['Summary Outstanding Receivables(USD)'] = np.nan
result4['Sales Contractual Counterparty Open Account Approved '] = np.nan
result4['Purchase Dry Weight'] = np.nan
result4['Purchase Dry Weight UOM'] = np.nan
result4['Purchase Wet Weight'] = np.nan
result4['Purchase Wet Weight UOM'] = np.nan
result4['Receipt In Progress'] = np.nan
result4['Delivery In Progress'] = np.nan
result4['Eligible Dry Weight'] = np.nan
result4['Eligible Wet Weight'] = np.nan

result4 = result4.drop(columns = ['%', 'status'])

In [126]:
result4 = result4[order4]
result4.reset_index(drop=True)
result4.shape[1]
# dff.shape[1]

51

In [122]:


final_result = pd.concat([dff, result4], ignore_index=True)
final_result = final_result.sort_values(by=['Sales Assignment Reference', 'Delivery In Progress'], ascending=[False, False], na_position= 'first') 
# final_result = final_result[order]
final_result = final_result[order4]
# final_result= final_result.sort_values(by=['Purchase Assignment Reference'], ascending=True)

# final_result.to_excel(path + "\output.xlsx", sheet_name='sales receivable')
# writer = pd.ExcelWriter(path + '/final_test.xlsx', engine='openpyxl')
# book = load_workbook(writer.path)
# writer.book = book
# final_result.to_excel(excel_writer=writer, sheet_name='sales receivables')
# writer.save()
# writer.close()

ValueError: Plan shapes are not aligned

In [37]:
def Assignment(a):
    return a == "649921.37.2"

df0 = final_sales.loc[final_sales['Purchase Assignment Reference'].apply(Assignment)]
df1 = nfinal_sales.loc[nfinal_sales['Purchase Assignment ID'].apply(Assignment)]
df2 = result4.loc[result4['Purchase Assignment Reference'].apply(Assignment)]
 
df2

,Purchase Group Company Code,Purchase Trade ID,Purchase Quota Titan ID,Purchase Assignment Reference,Sales Assignment Reference,Purchase Traffic Operator,Purchase Contractual Counterparty,Purchase Commodity,Sales Assignment Counterparty,Sales Credit Security Type,...,Purchase Dry Weight,Purchase Dry Weight UOM,Purchase Wet Weight,Purchase Wet Weight UOM,Receipt In Progress,Delivery In Progress,Eligible Dry Weight,Eligible Wet Weight,status,%
44,PTE,649921.0,649921.37,649921.37.2,670638.1.8,Alkistis Korali,Simar (Societa Metalli Marghera S.P.A.),Copper Blister,"Wuzhou Jinsheng Copper Co., Ltd",NaN,...,243.34,MT,243.34,MT,N,N,243.34,243.34,sales,0.499774
45,PTE,649921.0,649921.37,649921.37.2,670638.1.8,Alkistis Korali,Simar (Societa Metalli Marghera S.P.A.),Copper Blister,"Wuzhou Jinsheng Copper Co., Ltd",NaN,...,243.34,MT,243.34,MT,N,N,243.34,243.34,sales,0.499774


In [15]:


final_result.to_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0222\test_sales.xlsx')